# Item 36. 상속보다는 컴포지션을 사용하라

# 행위를 재사용하는 방법들
- 코드를 작성할때 DRY(Don't Repeat Yourself) 원칙을 따르는 것이 중요하다.
- 어떻게 반복작성을 줄일까?

## 함수 추출
- 가장 단순하고 쉬운방법
- 하지만 모든 중복을 추출만으로 제거할 수는 없다.
- 

In [ ]:
import java.io.BufferedReader
import java.io.FileReader

class CalculatorStep1Duplicated {
    fun sum(path: String): Int {
        var br: BufferedReader? = null
        try {
        br = BufferedReader(FileReader(path))
            var sum = 0
            var line: String?
            while (br.readLine().also { line = it } != null) {
                sum += line!!.toInt()
            }
            br.close()
            return sum
        } catch (e: Exception) {
            println("An error occurred. $e")
            throw e
        } finally {
            try {
            br?.close()
            } catch (e: Exception) {
                println("An error occurred. $e")
                throw e
            }
        }
    }

    fun multiply(path: String): Int {
        var br: BufferedReader? = null
        try {
            br = BufferedReader(FileReader(path))
            var sum = 1
            var line: String?
            while (br.readLine().also { line = it } != null) {
                sum *= line!!.toInt()
            }
            br.close()
            return sum
        } catch (e: Exception) {
            println("An error occurred. $e")
            throw e
        } finally {
            try {
                br?.close()
            } catch (e: Exception) {
                println("An error occurred. $e")
                throw e
            }
        }
    }
}

- 위와같은 코드는 중복코드가 앞뒤로 많기때문에 단순 함수 추출만으로는 해결불가능
  - 사실 람다를 넘기는 템플릿/콜백 패턴으로 가능하지만 지금 말하는 함수 추출은 단순 함수 추출만을 의미


## 상속과 템플릿 메서드 패턴
- 상속을 통해 중복을 제거하는 방법

In [ ]:
abstract class CalculatorTemplate {
    abstract val initialValue: Int

    fun calculate(path: String): Int {
        var br: BufferedReader? = null
        try {
            br = BufferedReader(FileReader(path))
            var result = initialValue
            var line: String?
            while (br.readLine().also { line = it } != null) {
                result = calculateInternal(result, line!!.toInt())
            }
            br.close()
            return result
        } catch (e: Exception) {
            println("An error occurred. $e")
            throw e
        } finally {
            try {
                br?.close()
            } catch (e: Exception) {
                println("An error occurred. $e")
                throw e
            }
        }
    }
    abstract fun calculateInternal(result: Int, number: Int) : Int

}

// 실제 구체 클래스의 모습
class PlusCalculator(override val initialValue: Int = 0) : CalculatorTemplate() {
    override fun calculateInternal(result: Int, number: Int): Int {
        return result + number
    }
}

class MultiplyCalculator(override val initialValue: Int = 1) : CalculatorTemplate() {
    override fun calculateInternal(result: Int, number: Int): Int {
        return result * number
    }
}

- 상속과 템플릿 메서드 패턴을 적용하면 위와같이 슈퍼클래스와 구체 클래스 정의로 중복되는 행위를 추출할수 있다.
- DRY!

### 문제점

1. 현재 자바, 코틀린은 단일 상속만 가능하므로 다양한 기능들을 구현하려면 복잡한 계층구조를 만들어야한다
2. 상속은 슈퍼 클래스의 모든것을 가져오기때문에, 불필요한 함수를 갖는 클래스가 만들어 질 수 있다.(인터페이스 분리가 적절하게 되지 못했기 떄문에 발생하는 문제이지만, 완벽하게 피하기가 쉽지 않다)
    ```java
       
    public abstract class AbstractDataSource implements DataSource {
    
        /** Logger available to subclasses. */
        protected final Log logger = LogFactory.getLog(getClass());
    
    
        /**
         * Returns 0, indicating the default system timeout is to be used.
         */
        @Override
        public int getLoginTimeout() throws SQLException {
            return 0;
        }
    
        /**
         * Setting a login timeout is not supported.
         */
        @Override
        public void setLoginTimeout(int timeout) throws SQLException {
            throw new UnsupportedOperationException("setLoginTimeout");
        }
    
        /**
         * LogWriter methods are not supported.
         */
        @Override
        public PrintWriter getLogWriter() {
            throw new UnsupportedOperationException("getLogWriter");
        }
    
        /**
         * LogWriter methods are not supported.
         */
        @Override
        public void setLogWriter(PrintWriter pw) throws SQLException {
            throw new UnsupportedOperationException("setLogWriter");
        }
    
    
        //---------------------------------------------------------------------
        // Implementation of JDBC 4.0's Wrapper interface
        //---------------------------------------------------------------------
    
        @Override
        @SuppressWarnings("unchecked")
        public <T> T unwrap(Class<T> iface) throws SQLException {
            if (iface.isInstance(this)) {
                return (T) this;
            }
            throw new SQLException("DataSource of type [" + getClass().getName() +
                    "] cannot be unwrapped as [" + iface.getName() + "]");
        }
    
        @Override
        public boolean isWrapperFor(Class<?> iface) throws SQLException {
            return iface.isInstance(this);
        }
    
    
        //---------------------------------------------------------------------
        // Implementation of JDBC 4.1's getParentLogger method
        //---------------------------------------------------------------------
    
        @Override
        public Logger getParentLogger() {
            return Logger.getLogger(Logger.GLOBAL_LOGGER_NAME);
        }
    
    }   
      
   ```
- 위의 예시처럼 javax의 dataSource 스펙이 과잉 명세화되어있어, 스프링에서 이를 구현하는 구현체를 만들때 패딩용 클래스를 중간에 정의했다.
- 3. 상속은 슈퍼 클래스의 코드를 읽어야한다는점에서 코드의 가독성을 저해한다

## 합성과 전략 패턴


In [ ]:
class CalculatorWithStrategy(
    private val strategies: List<CalculatorStrategy>,
) {
    fun sum(path: String): Int {
        return calculate(path, CalculateOperation.SUM)
    }

    fun multiply(path: String): Int {
        return calculate(path, CalculateOperation.MULTIPLY)
    }

    private fun calculate(path: String, operation: CalculateOperation): Int {
        var br: BufferedReader? = null
        try {
            br = BufferedReader(FileReader(path))
            val strategy = strategies.find { it.isSupport() == operation }
                ?: throw IllegalArgumentException("not supported operation. operation: $operation")
            var result = strategy.initValue()
            var line: String?
            while (br.readLine().also { line = it } != null) {
                result = strategy.calculate(result, line!!.toInt())
            }
            br.close()
            return result
        } catch (e: Exception) {
            println("An error occurred. $e")
            throw e
        } finally {
            try {
                br?.close()
            } catch (e: Exception) {
                println("An error occurred. $e")
                throw e
            }
        }
    }
}

enum class CalculateOperation {
    SUM, MULTIPLY
}

interface CalculatorStrategy {
    fun calculate(result: Int, number: Int): Int
    fun initValue(): Int
    fun isSupport(): CalculateOperation
}

class PlusStrategy : CalculatorStrategy {
    override fun calculate(result: Int, number: Int): Int {
        return result + number
    }

    override fun initValue(): Int {
        return 0
    }

    override fun isSupport(): CalculateOperation {
        return CalculateOperation.SUM
    }
}

class MultiplyStrategy : CalculatorStrategy {
    override fun calculate(result: Int, number: Int): Int {
        return result * number
    }

    override fun initValue(): Int {
        return 1
    }

    override fun isSupport(): CalculateOperation {
        return CalculateOperation.MULTIPLY
    }
}



- 위와같이 전략패턴을 사용할 경우, 가장 큰 차이점은 변경되는 코드를 가진 객체가 중복되는 코드를 가진 객체를 몰라도되고, 반대로 중복되는 코드를 가진 객체는 단순히 전략 인터페이스만 알면 된다는점
- 단순 상속에 비해 훨씬 약결합된 구조다.

### 합성의 장점
1. 상속보다 유연하다
    - 상속은 컴파일시점에 결정되는 반면, 합성은 런타임시점에 결정된다.
    - 이는 런타임시점에 객체를 교체할 수 있음을 의미한다.
2. 여러기능을 조합할 때, 합성은 복잡한 계층구조를 가지지 않고도 구현 가능하다.
    - 대표적으로 데코레이터 패턴 
3. 모든 메서드를 다 가져오는것이 아니므로 과잉명세를 피할 수 있다.
4. 상속은 캡슐화를 깨뜨릴 수 있지만, 합성은 캡슐화를 유지할 수 있다.

### 합성의 단점
- 다형성을 사용할 수 없다.

### 코틀린은 클래스 위임(class Delegation)으로 다형성있는 합성을 손쉽게 구현할 수 있다.
- 합성의단점인 다형성 문제는 위임패턴을 사용하면 어느정도 완화될 수 있다.
- 자바의 경우 위임패턴을 사용하기위해서는 다형성을 지원하는 껍데기, 포워딩 메서드를 가진 구현체가 필요하다
- 하지만 코틀린의 경우 by 키워드를 사용하여 위임을 쉽게 구현할 수 있다.

## 상속을 써야하는 상황 'is-a' 관계

- 상속은 명확한 'is-a' 관계를 나타낼때 사용해야한다.
- OOP 커뮤니티에서는 상속보다 합성이 낫다는게 전반적인 의견